In [ ]:
import glob
import pandas as pd
import numpy as np
import pickle
import re
from bs4 import BeautifulSoup
import collections
import requests
from itertools import chain
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
import string

In [ ]:
# init index list
tree_links_index = []
# Extract index pages
for i in range(1, 8):
    tree_links_index.append('http://www.llifle.com/Encyclopedia/TREES/Species/all/{0}/100/'.format(i))

# Init empty list
tree_links = []

for index_pages in tqdm(tree_links_index):
    # Extract XML
    URL = index_pages
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Extract links incomplete
    tree_links_half = soup.find_all('a')

    # Complete the links
    tree_links_temp = ['http://www.llifle.com' + pages.get('href') for pages in tree_links_half
                           if pages.get('href') != None 
                           if pages.get('href').startswith('/Encyclopedia/TREES/Family/')]
    # Add to all trees
    tree_links += tree_links_temp

In [ ]:
with open('../../data/processed/intermediate_web_llifle_treelist.pkl', 'wb') as f:
    pickle.dump(tree_links, f)

In [ ]:
# Init empty dict
data = collections.defaultdict(list)

for URL in tqdm(tree_links):

    # Get Page
    page = requests.get(URL)
    # Structure page
    soup = BeautifulSoup(page.content, 'html.parser')
    # Name
    name = soup.title.text.replace('\n', '')
    # Loop over text
    for text in soup.find_all('p'):
        try:
            if text.find('b').text.strip().startswith('Description'):
                data[name].append(tuple([1, text.text]))
            else:
                data[name].append(tuple([0, text.text]))
        except:
            continue

In [ ]:
with open('../../data/processed/descriptions_web_trees_llifle.pkl', 'wb') as f:
    pickle.dump(data, f)

# V2

In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
# init index list
links_index = []
# Extract TREES index pages
for i in range(1, 8):
    links_index.append('http://www.llifle.com/Encyclopedia/TREES/Species/all/{0}/100/'.format(i))
for i in range(1, 8):
    links_index.append('http://www.llifle.com/Encyclopedia/BROMELIADS/Species/all/{0}/100/'.format(i))    
for i in range(1, 16):
    links_index.append('http://www.llifle.com/Encyclopedia/PALMS%20AND%20CYCADS/Species/all/{0}/100/'.format(i))    
for i in range(1, 11):
    links_index.append('http://www.llifle.com/Encyclopedia/BULBS/Species/all/{0}/100/'.format(i)) 
for i in range(1, 179):
    links_index.append('http://www.llifle.com/Encyclopedia/CACTI/Species/all/{0}/100/'.format(i)) 
for i in range(1, 139):
    links_index.append('http://www.llifle.com/Encyclopedia/SUCCULENTS/Species/all/{0}/100/'.format(i))       
    
    
# Init empty list
links = []

for index_pages in tqdm_notebook(links_index[0:]):
    # Extract XML
    URL = index_pages
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    baselinks = soup.find_all('a')
    
    for baselink in baselinks:
        try:
            endlink = baselink['href']
            full_link = 'http://www.llifle.com' + endlink
            links.append(full_link)
        except:
            continue


In [ ]:
with open('../../data/processed/intermediate_web_llifle_links.pkl', 'wb') as f:
    pickle.dump(links, f)

In [ ]:
# Init empty dict
data = collections.defaultdict(list)

for URL in tqdm_notebook(links[0:]):
    try:
        # Get Page
        page = requests.get(URL)
        # Structure page
        soup = BeautifulSoup(page.content, 'html.parser')
        # Name
        name = soup.title.text.replace('\n', '')
        # Loop over text
        strings = [string for string in soup.stripped_strings if len(string.split()) > 10]
        for string in strings:
            doc = nlp(string)
            for sentence in doc.sents:
                data[name].append((URL, sentence.text))
    except:
        continue
    

In [ ]:
with open('../../data/processed/descriptions_llifeV2_PLANTS.pkl.pkl', 'wb') as f:
    pickle.dump(data, f)